
# Mini Transformer (Frases Curtas) – PyTorch

Este notebook demonstra, de forma **didática e leve**, como treinar um **mini Transformer** para **completar frases curtas** (nível de palavras) usando **PyTorch**.  
A proposta é construir um **modelo decoder-only simplificado** (usando `TransformerEncoder` com **máscara causal**) que aprende padrões básicos de sequência e consegue **prever a próxima palavra** a partir de um *prompt* curto.

**Você verá**:
- Como montar um **corpus mínimo** de frases curtas em PT-BR.
- Como construir um **vocabulário** e preparar tensores para treino.
- Como implementar um **MiniTransformerLM** (embedding + positional encoding + `TransformerEncoder` + cabeça linear).
- Como **treinar** rapidamente e **gerar texto** (autocompletar frases).

> **Observação didática**: este notebook é ideal para turmas de anos iniciais **entenderem o fluxo** (dados → modelo → treino → geração) e **desmistificar** o uso do Transformer em pequena escala.


In [ ]:
# !pip install torch --quiet  # Se estiver no Colab e precisar instalar
import math
import random
from typing import List, Dict

import re
import unicodedata

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
device



## 1. Corpus mínimo (frases curtas)

Vamos criar um conjunto **pequeno** de frases curtas em português.  
A ideia é permitir **treinos rápidos** (1–5 minutos) e ainda assim ver o modelo **aprender padrões** simples.


In [ ]:
corpus = [
    "o gato dorme",
    "o cachorro corre",
    "a menina sorri",
    "a menina corre",
    "o menino sorri",
    "o menino corre",
    "o gato mia",
    "o cachorro late",
    "a menina pula",
    "o menino pula",
    "o gato pula",
    "o cachorro dorme",
    "o aluno estuda",
    "a aluna estuda",
    "o professor explica",
    "a professora explica",
    "o aluno aprende",
    "a aluna aprende",
    "o livro cai",
    "a bola rola",
    "o carro anda",
    "o carro para",
    "a luz acende",
    "a luz apaga",
    "a criança brinca",
    "o bebê dorme",
]

# Embaralhar (opcional) para variar a ordem entre execuções
random.seed(42)
random.shuffle(corpus)

len(corpus), corpus[:5]


## 2. Tokenização e Vocabulário (nível de palavra)

Usaremos uma **tokenização simples por espaço** e adicionaremos tokens especiais:
- `<pad>`: padding (alinhamento dos comprimentos)
- `<bos>`: início de sequência
- `<eos>`: final de sequência

O modelo receberá como entrada `bos + tokens_da_frase` e aprenderá a **prever o próximo token** em cada passo (linguagem autoregressiva).


In [ ]:
PAD, BOS, EOS, UNK = "<pad>", "<bos>", "<eos>", "<unk>"

def normalize_text(s: str) -> str:
    # lowercase
    s = s.lower().strip()
    # opcional: remover acentos (comente se não quiser)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    # separar pontuação simples (mantém vírgulas/pontos como tokens)
    s = re.sub(r"([,.;:!?])", r" \1 ", s)
    # colapsar espaços
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokenize(sentence: str) -> List[str]:
    return normalize_text(sentence).split()

# Reconstruir corpus normalizado (mantém o texto original apenas para referência)
corpus_norm = [normalize_text(s) for s in corpus]

# Construir vocabulário com <unk>
vocab_set = {PAD, BOS, EOS, UNK}
for s in corpus_norm:
    vocab_set.update(tokenize(s))

itos = sorted(list(vocab_set))               # index -> string
stoi: Dict[str, int] = {tok: i for i, tok in enumerate(itos)}  # string -> index
vocab_size = len(itos)

itos[:10], vocab_size


## 3. Dataset e DataLoader

Vamos preparar tensores de entrada e saída.  
Dado um exemplo `"o gato dorme"`, o **input** será `"<bos> o gato dorme"` e o **target** será `"o gato dorme <eos>"`.  
Assim, o modelo aprende a prever o **próximo token** a cada posição (treino autoregressivo).


In [ ]:
def encode(tokens: List[str]) -> List[int]:
    return [stoi.get(t, stoi[UNK]) for t in tokens]

def detok(ids: List[int]) -> str:
    toks = [itos[i] for i in ids if i != stoi[PAD]]
    # remove <bos> e corta em <eos>
    try:
        if BOS in toks:
            toks = toks[toks.index(BOS)+1:]
    except ValueError:
        pass
    try:
        if EOS in toks:
            toks = toks[:toks.index(EOS)]
    except ValueError:
        pass
    return " ".join(toks)

class TinyLMDataset(Dataset):
    def __init__(self, sentences: List[str], max_len: int = None):
        self.examples = []
        tokenized = [tokenize(s) for s in sentences]
        if max_len is None:
            max_len = max(len(toks) for toks in tokenized) + 1  # +1 para BOS/EOS
        self.max_len = max_len

        for toks in tokenized:
            inp = [BOS] + toks
            tgt = toks + [EOS]
            self.examples.append((inp, tgt))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        inp, tgt = self.examples[idx]
        return inp, tgt

def pad_to_len(ids: List[int], max_len: int, pad_id: int) -> List[int]:
    return ids + [pad_id] * (max_len - len(ids))

def collate_fn(batch):
    pad_id = stoi[PAD]
    xs, ys = [], []
    max_len = max(len(x) for x, _ in batch)
    for inp, tgt in batch:
        x_ids = encode(inp)
        y_ids = encode(tgt)
        x_ids = pad_to_len(x_ids, max_len, pad_id)
        y_ids = pad_to_len(y_ids, max_len, pad_id)
        xs.append(x_ids)
        ys.append(y_ids)
    return torch.tensor(xs, dtype=torch.long), torch.tensor(ys, dtype=torch.long)

# Use o corpus normalizado aqui:
dataset = TinyLMDataset(corpus_norm)
loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

batch_x, batch_y = next(iter(loader))
batch_x.shape, batch_y.shape, batch_x[0], batch_y[0]



## 4. Modelo: MiniTransformerLM

Arquitetura **simples e didática**:
- **Embedding** de tokens
- **Positional Encoding senoidal**
- **TransformerEncoder** (com **máscara causal** para não olhar o futuro)
- **Camada linear** para prever distribuição de probabilidade no vocabulário


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch, seq_len, d_model)
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

def generate_causal_mask(sz: int) -> torch.Tensor:
    # Máscara triangular superior para impedir atenção ao futuro
    mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
    return mask

class MiniTransformerLM(nn.Module):
    def __init__(self, vocab_size: int, d_model=128, nhead=4, num_layers=2, dim_feedforward=256, dropout=0.1):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=256)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # x: (batch, seq_len)
        emb = self.tok_emb(x)                 # (batch, seq_len, d_model)
        h = self.pos_enc(emb)                 # + posição
        seq_len = x.size(1)
        mask = generate_causal_mask(seq_len).to(x.device)  # (seq_len, seq_len)
        h = self.encoder(h, mask=mask)        # (batch, seq_len, d_model)
        logits = self.lm_head(h)              # (batch, seq_len, vocab_size)
        return logits



## 5. Treinamento rápido

Usaremos **CrossEntropyLoss** ignorando `<pad>` e um otimizador **Adam**.  
Como o corpus é pequeno, poucas épocas já mostram aprendizado de padrões.


In [ ]:
def train_model(model, loader, epochs=10, lr=3e-3):
    model = model.to(device)
    pad_id = stoi[PAD]
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    losses = []

    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        steps = 0
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)  # (B, T, V)
            B, T, V = logits.shape
            loss = criterion(logits.view(B*T, V), y.view(B*T))

            optim.zero_grad()
            loss.backward()
            optim.step()

            total_loss += loss.item()
            steps += 1

        avg_loss = total_loss / steps
        losses.append(avg_loss)
        print(f"Epoch {epoch:02d} | loss: {avg_loss:.4f}")
    return losses

model = MiniTransformerLM(vocab_size=vocab_size, d_model=128, nhead=4, num_layers=2, dim_feedforward=256)
losses = train_model(model, loader, epochs=200, lr=3e-3)



### Gráfico de perda (loss)


In [ ]:
plt.figure()
plt.plot(losses)
plt.title("Treinamento - Loss por época")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.show()



## 6. Geração: completar frases

Função de **decodificação autoregressiva** (greedy) a partir de um *prompt* inicial.  
Você pode testar com entradas como `"o cachorro"`, `"a menina"`, `"o aluno"`, etc.


In [ ]:
def detok(ids: List[int]) -> str:
    toks = [itos[i] for i in ids if i != stoi[PAD]]
    # Remove <bos> se aparecer e corta em <eos>
    if BOS in toks:
        toks = toks[toks.index(BOS)+1:] if BOS in toks else toks
    if EOS in toks:
        toks = toks[:toks.index(EOS)]
    return " ".join(toks)

@torch.no_grad()
def generate(model, prompt: str, max_new_tokens=5):
    model.eval()
    toks = tokenize(prompt)
    if not toks:
        toks = [UNK]
    in_tokens = [BOS] + toks
    x = torch.tensor([encode(in_tokens)], dtype=torch.long).to(device)

    for _ in range(max_new_tokens):
        logits = model(x)  # (1, T, V)
        next_token_logits = logits[:, -1, :]
        next_id = torch.argmax(next_token_logits, dim=-1)
        x = torch.cat([x, next_id.unsqueeze(0)], dim=1)
        if next_id.item() == stoi[EOS]:
            break
    return detok(x[0].tolist())

# Exemplos rápidos
tests = ["o cachorro", "a menina", "o gato", "o aluno", "a luz"]
for t in tests:
    print(f"Entrada: {t!r} -> Saída: {generate(model, t, max_new_tokens=5)!r}")



## 7. Extensões sugeridas

- **Aumentar o corpus** com mais frases e variações verbais.
- **Ajustar o tamanho do modelo** (`d_model`, `nhead`, `num_layers`) para observar limites de capacidade.
- **Trocar greedy por amostragem** (*top-k*/*top-p*) para sentenças mais diversas.
- **Adicionar regularização** (dropout maior) para evitar overfitting no corpus minúsculo.
- **Hiperparâmetros**: teste `lr`, `epochs`, `batch_size` e observe o impacto no gráfico de *loss*.
